In [1]:
import sqlite3

# Create SQLite database
conn = sqlite3.connect("company.db")
cursor = conn.cursor()

# Create Employees table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Employees (
    ID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Department TEXT NOT NULL,
    Salary INTEGER NOT NULL,
    Hire_Date TEXT NOT NULL
)
""")

# Create Departments table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Departments (
    ID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Manager TEXT NOT NULL
)
""")

# Insert sample data
cursor.executemany("INSERT OR IGNORE INTO Employees VALUES (?, ?, ?, ?, ?)", [
    (1, 'Alice', 'Sales', 50000, '2021-01-15'),
    (2, 'Bob', 'Engineering', 70000, '2020-06-10'),
    (3, 'Charlie', 'Marketing', 60000, '2022-03-20')
])

cursor.executemany("INSERT OR IGNORE INTO Departments VALUES (?, ?, ?)", [
    (1, 'Sales', 'Alice'),
    (2, 'Engineering', 'Bob'),
    (3, 'Marketing', 'Charlie')
])

conn.commit()
conn.close()

print("Database setup complete!")


Database setup complete!


In [13]:
import re

def extract_department(query, pattern):
    match = re.search(pattern, query)
    return match.group(1).strip() if match else None





def process_query(query):
    conn = sqlite3.connect("company.db")
    cursor = conn.cursor()
    query = query.lower()

    if "employees in the" in query:
        dept = extract_department(query, r"employees in the (.*?) department")
        if dept:
            cursor.execute("SELECT Name FROM Employees WHERE LOWER(Department) = LOWER(?)", (dept,))
            results = cursor.fetchall()
            response = ", ".join([row[0] for row in results]) if results else "No employees found."
        else:
            response = "Invalid department format."

    elif "manager of the" in query:
        dept = extract_department(query, r"manager of the (.*?) department")
        if dept:
            cursor.execute("SELECT Manager FROM Departments WHERE LOWER(Name) = LOWER(?)", (dept,))
            result = cursor.fetchone()
            response = result[0] if result else "Department not found."
        else:
            response = "Invalid department format."

    elif "hired after" in query:
        date = query.split("hired after ")[-1]
        cursor.execute("SELECT Name FROM Employees WHERE Hire_Date > ?", (date,))
        results = cursor.fetchall()
        response = ", ".join([row[0] for row in results]) if results else "No employees found."

    elif "total salary expense for the" in query:
        dept = extract_department(query, r"total salary expense for the (.*?) department")
        if dept:
            cursor.execute("SELECT SUM(Salary) FROM Employees WHERE LOWER(Department) = LOWER(?)", (dept,))
            result = cursor.fetchone()
            response = f"Total salary expense: {result[0]}" if result and result[0] else "Department not found or no employees."
        else:
            response = "Invalid department format."

    else:
        response = "Sorry, I don't understand that query."

    conn.close()
    return response


In [14]:
query1 = "Show me all employees in the Sales department."
query2 = "Who is the manager of the Marketing department?"
query3 = "List all employees hired after 2021-01-01."
query4 = "What is the total salary expense for the Engineering department?"

print("Query 1:", process_query(query1))  # Should return: Alice
print("Query 2:", process_query(query2))  # Should return: Charlie
print("Query 3:", process_query(query3))  # Should return: Alice, Charlie
print("Query 4:", process_query(query4))  # Should return: Total salary expense: 70000


Query 1: Alice
Query 2: Charlie
Query 3: Alice, Charlie
Query 4: Total salary expense: 70000
